**Setup environment and dependencies**

Note: This notebook is experted from kaggle personal account

    - pip install uv
    - pyenv to manage python version
    - using 3.11
    - uv sync
    - unzip data


In [ ]:
!uv add numpy pandas 
!uv add nltk gensim
!uv add scikit-learn


In [ ]:
# Using gensim -> True
# to use nltk stemming set False
USE_GENSIM=True
# USE_GENSIM=False

# gensim takes 0.6 s for me and nltk took 2.5 test twice


In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Steps
**Short description**
- Load datasets into DF using pd
    - Explore datasets
    - shape
    - columns info()
    - datatypes
    - head and head(count)
- combine dataframes into one using merge
    - droped title from credits and used only movie_id, inplace=True
    - merge(on=Column name)
    - merge left_on, right_on used becuase i used id and movie_id and with innner as how
- feature selection
  - Null checks
  - duplicate checks
  - drop null
- data preprocessing
- Vectorization
- Main function to return movies
- Saving Model
---

### Step 1

- Load datasets into DF using pd
    - Explore datasets
    - shape
    - columns info()
    - datatypes
    - head and head(count)

In [ ]:
movies = pd.read_csv('data/tmdb_5000_movies.csv')
credits = pd.read_csv('data/tmdb_5000_credits.csv')


In [ ]:
print("Movies DF shape:", movies.shape, "Credits DF shape:", credits.shape)

In [ ]:
movies.info()

In [ ]:
credits.info()

In [ ]:
movies.head(2)

In [ ]:
credits.head(2)

### Step 2
- combine dataframes into one using merge
    - droped title from credits and used only movie_id, inplace=True
    - merge(on=Column name)
    - merge left_on, right_on used becuase i used id and movie_id and with innner as how

In [ ]:
movies.merge?

In [ ]:
credits.drop('title', axis=1, inplace=True)

In [ ]:
movies = movies.merge(credits, left_on=['id'], right_on=['movie_id'])

In [ ]:
movies.head(1)

---
### Step 3
- feature selection
  - Null checks
  - duplicate checks
  - drop null

**Selected Feature**
* genre
* keywords
* title -> english
* overview
* cast -> name -> Actors real name -> top 3
* crew -> Director

In [ ]:
_movies = movies[['id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
_movies.head(3)

In [ ]:
_movies.isna().sum()

In [ ]:
movies = _movies.dropna()
movies.isna().sum()

In [ ]:
movies.duplicated().sum()

---
### Step 4

- data preprocessing
  - Cleanup for columns like genres, keywords, cast, crew
  - Spaces cleanup
  - Tags combined column

In [ ]:
movies['genres'][0]

# reading 0 index will return string
movies['genres'][0][0]

In [ ]:
# temp testing
import json
temp= json.loads(movies['genres'][0])

print("data at 0", temp[0], "\nname at 0", temp[0]['name'])

# or use ast.literal_eval
# import ast

In [ ]:
def data_parser(data_obj, data_key='name'):
    d_list = []
    try:
        data = json.loads(data_obj)
        for item in data:
            d_list.append(item[data_key])   
    except Exception:
        print("Error")
        d_list.append(None)
    return d_list

In [ ]:
movies.head(1)

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x: data_parser(x))

In [ ]:
movies['keywords'] = movies['keywords'].apply(lambda x: data_parser(x))


In [ ]:
movies.head(1)

In [ ]:
def cast_parser(data_obj, data_key='name', limit=3):
    d_list = []
    try:
        data = json.loads(data_obj)
        data = data[:limit]
        for item in data:
            d_list.append(item[data_key])   
    except Exception:
        print("Error")
        d_list.append(None)
    return d_list

In [ ]:
movies['cast'] = movies['cast'].apply(lambda x: cast_parser(x))

In [ ]:
movies.shape

In [ ]:
movies['crew'][0]

In [ ]:
def director_crew_parser(data_obj, data_key='name'):
    d_list = []
    try:
        data = json.loads(data_obj)
        for item in data:
            if item['job'] == 'Director':
                d_list.append(item[data_key])   
                break
    except Exception:
        print("Error")
        d_list.append(None)
    return d_list

In [ ]:
movies['crew'] = movies['crew'].apply(lambda x: director_crew_parser(x))

In [ ]:
movies.isna().sum()

In [ ]:
movies.head()

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [ ]:
movies.head()

In [ ]:
movies.loc[:,'genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies.loc[:,'keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies.loc[:,'cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies.loc[:,'crew'] = movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])


In [ ]:
movies.head()

In [ ]:
movies.loc[:, 'tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
main_df = movies[['id', 'title', 'tags']]

In [ ]:
main_df

In [ ]:
main_df['tags'] = main_df['tags'].apply(lambda x: " ".join(x))

In [ ]:
main_df['tags'][0]

In [ ]:
# we can leverage CountVectoriser to apply lowercase
main_df['tags'] = main_df['tags'].apply(lambda x: x.lower())


In [ ]:
main_df

### Step 5

Vectorization

* Text vectorization
    * Text similarity between tags
    * Stop words removal
    * 5000 minimal words for now

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(main_df['tags'])

In [ ]:
cv.get_feature_names_out()[-5:]

In [ ]:
vectors.shape

In [ ]:
if USE_GENSIM:
    from gensim.parsing.porter import PorterStemmer
    ps = PorterStemmer()
    print("gensim")
else:
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    print("nltk")

In [ ]:
def stem(input:str)->str:
    out = [ps.stem(i) for i in input.split()]
    return " ".join(out)
    

In [ ]:
stem(main_df['tags'][0])

In [ ]:
# Gensim
# 'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, 
# but becom torn between follow order and protect an alien civilization. action adventur fantasi
# sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi 
# marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [ ]:
# nltk
# 'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission,
# but becom torn between follow order and protect an alien civilization. action adventur fantasi
# sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi
# marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [ ]:
main_df.loc[:, 'tags'] = main_df['tags'].apply(stem)

In [ ]:
vectors = cv.fit_transform(main_df['tags'])

In [ ]:
cv.get_feature_names_out()[:40]

In [ ]:
vectors.shape

### Step 6

Similarity between movies to other movies
some chart like heatmap
* Create similarity matrix
* cosine similarity -> uses angles
* using indexes to get similarity matrix
* return movie data using index and main df

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)


In [ ]:
similarity.shape

In [ ]:
print(main_df[main_df['title']=='Avatar'])

In [ ]:
def recommend_movie(movie_name):
    # get movie data from df matching to title
    # movie = main_df[main_df['title'] == movie_name]

    # using lowercase for better comparision
    movie = main_df[main_df["title"].str.lower() == movie_name.lower()]

    if movie.empty:
        out = {"status": "success", "data": []}
        return out

    # get index to so similarity matrix can be used to get similarity based of index
    movie_index = movie.index[0]

    # get similarity matrix of this movie using movie_index
    movie_similarity_matrix = similarity[movie_index]

    # need to sort to get top highest similarity based movies and keeping the indexes at same place or maybe
    # known
    closest_movies_sorted = sorted(
        enumerate(movie_similarity_matrix), reverse=True, key=lambda x: x[1]
    )
    similar_movies_five: list[tuple[int, np.float64]] = closest_movies_sorted[1:6]
    # [(1213, np.float64(0.2847987184339659))] -> [(index, similarity)]

    recommended_movies = [
        {
            "movie_id": main_df.loc[item[0]].id,
            "movie_name": main_df.loc[item[0]].title,
            "movie_index": item[0],
            "movie_similarity": item[1],
        }
        for item in similar_movies_five
    ]

    out = {"status": "success", "data": recommended_movies}

    return out

In [ ]:
recommend_movie('Moon knight')

In [ ]:
def recommend_movie_by_id(movie_id):
    # get movie data from df matching to id

    # using lowercase for better comparision
    movie = main_df[main_df["id"] == movie_id]

    if movie.empty:
        out = {"status": "success", "data": []}
        return out

    # get index to so similarity matrix can be used to get similarity based of index
    movie_index = movie.index[0]

    # get similarity matrix of this movie using movie_index
    movie_similarity_matrix = similarity[movie_index]

    # need to sort to get top highest similarity based movies and keeping the indexes at same place or maybe
    # known
    closest_movies_sorted = sorted(
        enumerate(movie_similarity_matrix), reverse=True, key=lambda x: x[1]
    )
    similar_movies_five: list[tuple[int, np.float64]] = closest_movies_sorted[1:6]
    # [(1213, np.float64(0.2847987184339659))] -> [(index, similarity)]

    recommended_movies = [
        {
            "movie_id": main_df.loc[item[0]].id,
            "movie_name": main_df.loc[item[0]].title,
            "movie_index": item[0],
            "movie_similarity": item[1],
        }
        for item in similar_movies_five
    ]

    out = {"status": "success", "data": recommended_movies}

    return out

In [ ]:
recommend_movie_by_id(19995)

### Step 7

- backup for later use

In [ ]:
import joblib
movies_df = main_df[['id', 'title']]
movies_df.sort_values(by='title', ascending=True, inplace=True)
joblib.dump((movies_df, similarity), 'movies_recommendations_assets_min.joblib')


In [ ]:
df, similarity_matrix = joblib.load("movie_recommendation_assets.joblib")


In [ ]:
_df = df[['id', 'title']]

In [ ]:
# def recommend_movies():
#     movie_id = '19995'
#     main_df, similarity = joblib.load("movie_recommendation_assets.joblib")

#     # get movie data from df matching to id
#     movie = main_df[main_df["id"] == int(movie_id) ]

#     if movie.empty:
#         out = {"status": "success", "data": []}
#         return out

#     # get index to so similarity matrix can be used to get similarity based of index
#     movie_index = movie.index[0]

#     # get similarity matrix of this movie using movie_index
#     movie_similarity_matrix = similarity[movie_index]

#     # need to sort to get top highest similarity based movies and keeping the indexes at same place or maybe
#     # known
#     closest_movies_sorted = sorted(
#         enumerate(movie_similarity_matrix), reverse=True, key=lambda x: x[1]
#     )
#     similar_movies_five: list[tuple[int, np.float64]] = closest_movies_sorted[1:6]
#     # [(1213, np.float64(0.2847987184339659))] -> [(index, similarity)]

#     recommended_movies = [
#         {
#             "movie_id": main_df.loc[item[0]].id,
#             "movie_name": main_df.loc[item[0]].title,
#             "movie_index": item[0],
#             "movie_similarity": item[1],
#         }
#         for item in similar_movies_five
#     ]

#     out = {"status": "success", "data": recommended_movies}

#     return out


In [ ]:
# recommend_movies()